In [1]:
# Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16, DenseNet201
from tensorflow.keras.models import Model
import os
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import label_binarize


import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperParameters 


# ----------- CONSTANTS ----------------

# define directory structure
TRAIN_DIR = "PROCESSED_DATA/TRAINING_DATA/TRAINING_AUGMENTED_DATA"
VALID_DIR = "PROCESSED_DATA/VALIDATION_DATA/"
TEST_DIR = "PROCESSED_DATA/TEST_DATA/"

# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NORMALIZE_FLAG = True
NO_FRILLS_DATAGEN = ImageDataGenerator()
NORM_DATAGEN = ImageDataGenerator(rescale=1./255)


def load_data(directory,shuffle_flag=True):
    '''
    Param: 
        - directory - str, 
        - shuffle_flag - boolean, introduces constrolled stochasticity
    '''
    if NORMALIZE_FLAG == True:
        generator = NORM_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator
    else:
        generator = NO_FRILLS_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator

TRAIN_GENERATOR = load_data(TRAIN_DIR)
VAL_GENERATOR = load_data(VALID_DIR)
TEST_GENERATOR = load_data(TEST_DIR,shuffle_flag=False)
hp = HyperParameters()

def build_transfer_learning(hp):
    '''
    builds a transfer learning model using denseNet201 with classification layers
    specified in the JutePestDetect paper: https://arxiv.org/pdf/2308.05179 
    
    classification portion includes:
    -global average pooling layer
    -instead of 30% dropout layer, using tunable dropout
    -dense layer w softmax classifier
    '''
    base_model = DenseNet201(weights='imagenet', include_top=False,input_shape=(224, 224, 3))
    
    # freeze the base model layers
    base_model.trainable = False
    
    dropout_rate = hp.Float("dropout", min_value=0.2, max_value=0.5, step=0.1)
    
    # add global average pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = Dropout(dropout_rate)(x)

    # dense layer w softmax classifier
    output_layer = Dense(TRAIN_GENERATOR.num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output_layer)
    
    # tune
    learning_rate = hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
    batch_size = hp.Choice('batch_size', values=[16, 32])
    optimizer = Adam(learning_rate=learning_rate)
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    

    return model

def build_best_model_transfer_learning():

    # Define the Bayesian tuner
    tuner = kt.BayesianOptimization(
        build_transfer_learning,
        objective='val_accuracy',  # tune by improving validation accuracy
        max_trials=20,  # num different hp combos to try
        executions_per_trial=1,  # run each model once
        directory='bayesian_tuning',
        project_name='lr_and_drop_tuning_DenseNet201'
    )
    
    # search hp combos
    tuner.search(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=10)
    
    # get best hps
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    # save them
    best_hps_dict = {'best_lr': best_hps.get('lr'),
                     'best_dropout': best_hps.get('dropout'),
                     'best_batch_size': best_hps.get('batch_size')}
    
    # make final model with the best drop out, learning rate and batch size
    best_model = tuner.hypermodel.build(best_hps)
    best_model_training_history = best_model.fit(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=10, batch_size=best_hps.get('batch_size'))
    
    return best_hps_dict, best_model, best_model_training_history


def evaluate_model(model, filename = "best_model_densenet201.h5"):
    '''
    Saves model to h5 file, returns test accuracy loss and test accuracy
    '''
    # evaluate on test data
    test_loss, test_acc = model.evaluate(TEST_GENERATOR)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # save to file
    model.save(filename)

    return test_loss, test_acc


def main():

    # best model
    best_hps_dict, best_model, best_model_training_history = build_best_model_transfer_learning()

    print(f'best parameters:\n {best_hps_dict}')
    
    test_loss, test_acc = evaluate_model(best_model)
    print(f'test_loss: {test_loss}, test_acc: {test_acc}')
    
    best_model.save("densenet201_best_model_bayes_optimization.h5")
    
if __name__ == "__main__":
    main()


Trial 20 Complete [00h 29m 53s]
val_accuracy: 0.7840909361839294

Best val_accuracy So Far: 0.7909091114997864
Total elapsed time: 10h 10m 52s
Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 184s 857ms/step - accuracy: 0.4590 - loss: 1.6527 - val_accuracy: 0.7409 - val_loss: 0.8454
Epoch 2/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 850ms/step - accuracy: 0.7905 - loss: 0.6823 - val_accuracy: 0.7500 - val_loss: 0.7995
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 851ms/step - accuracy: 0.8312 - loss: 0.5310 - val_accuracy: 0.7614 - val_loss: 0.7649
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 852ms/step - accuracy: 0.8715 - loss: 0.4386 - val_accuracy: 0.7682 - val_loss: 0.7260
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 851ms/step - accuracy: 0.8767 - loss: 0.3996 - val_accuracy: 0.7864 - val_loss: 0.7439
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 851ms/step - accuracy: 0.8911 - loss: 0.3490 - val_accuracy: 0.7795 - val_loss: 0.7329
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 177s 852ms/step - accurac

/Users/shirleyfong/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 789ms/step - accuracy: 0.7873 - loss: 0.6405


Test Accuracy: 0.7727


test_loss: 0.681067705154419, test_acc: 0.7727272510528564
